In [2]:
import pandas as pd
import numpy as np

# Data loading

### Expression levels

In [3]:
brca_ex = pd.read_hdf('../data/TCGA_gene_exp_20k_std-MAD.h5', key='brca')
non_brca_ex = pd.read_hdf('../data/TCGA_gene_exp_20k_std-MAD.h5', key='non_brca')

In [4]:
all_ex = pd.concat([brca_ex, non_brca_ex])

In [5]:
all_ex.shape

(10497, 20000)

In [6]:
import ast
with open('../common_genes_1-5.txt','r') as f:
   common_genes = ast.literal_eval(f.read())

In [7]:
all_ex = all_ex[list(common_genes)]

In [8]:
all_ex.shape

(10497, 1492)

corr = all_ex.corr()

In [9]:
non_collinear_genes = pd.read_csv('non_collinear_genes.csv')
non_collinear_genes = non_collinear_genes[non_collinear_genes.columns[1:]]

In [10]:
non_collinear_genes = list(non_collinear_genes.values[0])

In [11]:
all_ex = all_ex[non_collinear_genes]

In [13]:
all_ex.shape

(10497, 149)

### OS

In [15]:
brca_os = pd.read_hdf('../data/TCGA_data.h5', key='brca_clinical')[['OS', 'OS.time']]
non_brca_os = pd.read_hdf('../data/TCGA_data.h5', key='non_brca_clinical')[['OS', 'OS.time']]

In [16]:
## brca
brca_os['OS.time'] = brca_os['OS.time'].map(lambda x: np.nan if x == 'NaN' else x)
brca_os['OS'] = brca_os['OS'].map(lambda x: np.nan if x == 'NaN' else x)
brca_os.dropna(subset=['OS.time', 'OS'], inplace=True)
brca_os['OS.time'] = brca_os['OS.time'].astype(float)
brca_os['OS'] = brca_os['OS'].astype(float)

## non_brca
non_brca_os['OS.time'] = non_brca_os['OS.time'].map(lambda x: np.nan if x == 'NaN' else x)
non_brca_os['OS'] = non_brca_os['OS'].map(lambda x: np.nan if x == 'NaN' else x)
non_brca_os.dropna(subset=['OS.time', 'OS'], inplace=True)
non_brca_os['OS.time'] = non_brca_os['OS.time'].astype(float)
non_brca_os['OS'] = non_brca_os['OS'].astype(float)

In [18]:
all_os = pd.concat([brca_os, non_brca_os])

In [19]:
all_os.shape

(10420, 2)

In [20]:
all_ex = all_ex.loc[all_os.index]

In [22]:
sum(all_ex.index == all_os.index)

10420

In [25]:
all_data = pd.concat([all_ex, all_os], axis=1, join_axes=[all_ex.index])

In [26]:
all_data.shape

(10420, 151)

In [28]:
from lifelines import CoxPHFitter

In [34]:
cph = CoxPHFitter()
cph.fit(all_data, duration_col='OS.time', event_col='OS', show_progress=True)

cph.print_summary()  # access the results using cph.summary

Iteration 1: norm_delta = 0.95276, step_size = 0.9500, ll = -27378.61690, newton_decrement = 1108.50247, seconds_since_start = 1.3
Iteration 2: norm_delta = 0.25154, step_size = 0.9500, ll = -26520.33696, newton_decrement = 174.17382, seconds_since_start = 2.8
Iteration 3: norm_delta = 0.08235, step_size = 0.9500, ll = -26326.70716, newton_decrement = 22.29031, seconds_since_start = 3.8
Iteration 4: norm_delta = 0.03527, step_size = 1.0000, ll = -26301.74983, newton_decrement = 2.07548, seconds_since_start = 5.2
Iteration 5: norm_delta = 0.00469, step_size = 1.0000, ll = -26299.53966, newton_decrement = 0.02407, seconds_since_start = 6.6
Iteration 6: norm_delta = 0.00007, step_size = 1.0000, ll = -26299.51539, newton_decrement = 0.00000, seconds_since_start = 8.0
Iteration 7: norm_delta = 0.00000, step_size = 1.0000, ll = -26299.51538, newton_decrement = 0.00000, seconds_since_start = 9.3
Convergence completed after 7 iterations.
<lifelines.CoxPHFitter: fitted with 10420 observations, 

Concordance = 0.74
Log-likelihood ratio test = 2158.20 on 149 df, -log2(p)=inf


In [35]:
cph.check_assumptions(all_data, p_value_threshold=0.05, show_plots=True)


The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some covariates
will be below the threshold (i.e. by chance). This is compounded when there are many covariates.

Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and eyeball tests to
determine the most serious violations.


<lifelines.StatisticalResult>
         test_name = proportional_hazard_test
 null_distribution = chi squared
degrees_of_freedom = 1

---
                         test_statistic      p  -log2(p)
ENSG00000004468.12 km              0.87   0.35      1.51
                   rank            0.67   0.41      1.28
ENSG00000005073.5  km              0.26   0.61      0.72
                   rank            1.21   0.27      1.89
ENSG00000005844.17 km              0.00   0.98      0.03
                   rank            0.01   0.93      0.1

TypeError: 'TimeTransformers' object is not iterable

In [33]:
from lifelines.statistics import proportional_hazard_test
results = proportional_hazard_test(cph, all_data, time_transform='rank')
results.print_summary(decimals=3, model="untransformed variables")

<lifelines.StatisticalResult>
         test_name = proportional_hazard_test
    time_transform = rank
 null_distribution = chi squared
degrees_of_freedom = 1
             model = untransformed variables

---
                    test_statistic       p  -log2(p)
ENSG00000004468.12           0.670   0.413     1.276
ENSG00000005073.5            1.214   0.271     1.886
ENSG00000005844.17           0.008   0.930     0.104
ENSG00000008394.12           4.259   0.039     4.679
ENSG00000008517.16           0.048   0.827     0.274
ENSG00000037280.15           2.693   0.101     3.310
ENSG00000038945.14           2.015   0.156     2.683
ENSG00000039068.18           3.330   0.068     3.877
ENSG00000048052.21           0.005   0.945     0.082
ENSG00000049283.17           0.797   0.372     1.427
ENSG00000050555.17           0.014   0.907     0.141
ENSG00000051180.16           3.762   0.052     4.254
ENSG00000058091.16           0.662   0.416     1.266
ENSG00000065325.12           0.204   0.652     0.6

In [47]:
 

cph2 = CoxPHFitter()
cph2.fit(all_data, strata=['ENSG00000138622.3'], duration_col='OS.time', event_col='OS', show_progress=True)

cph2.print_summary()  # access the results using cph.summary

Iteration 1: norm_delta = 0.91802, step_size = 0.9500, ll = -17518.90775, newton_decrement = 994.78293, seconds_since_start = 2.0
Iteration 2: norm_delta = 0.23111, step_size = 0.9500, ll = -16722.40117, newton_decrement = 150.26055, seconds_since_start = 4.1
Iteration 3: norm_delta = 0.08146, step_size = 0.9500, ll = -16557.72443, newton_decrement = 13.50093, seconds_since_start = 6.2
Iteration 4: norm_delta = 0.02475, step_size = 1.0000, ll = -16543.03138, newton_decrement = 0.72287, seconds_since_start = 8.3
Iteration 5: norm_delta = 0.00162, step_size = 1.0000, ll = -16542.28241, newton_decrement = 0.00241, seconds_since_start = 10.2
Iteration 6: norm_delta = 0.00001, step_size = 1.0000, ll = -16542.28000, newton_decrement = 0.00000, seconds_since_start = 12.2
Convergence completed after 6 iterations.
<lifelines.CoxPHFitter: fitted with 10420 observations, 7169 censored>
      duration col = 'OS.time'
         event col = 'OS'
            strata = ['ENSG00000138622.3']
number of su

In [48]:
cph2.check_assumptions(all_data, p_value_threshold=0.05, show_plots=True)


The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some covariates
will be below the threshold (i.e. by chance). This is compounded when there are many covariates.

Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and eyeball tests to
determine the most serious violations.


<lifelines.StatisticalResult>
         test_name = proportional_hazard_test
 null_distribution = chi squared
degrees_of_freedom = 1

---
                         test_statistic      p  -log2(p)
ENSG00000004468.12 km              0.37   0.54      0.88
                   rank            0.44   0.51      0.98
ENSG00000005073.5  km              0.27   0.60      0.73
                   rank            5.05   0.02      5.34
ENSG00000005844.17 km              0.21   0.65      0.63
                   rank            0.11   0.74      0.4

TypeError: 'TimeTransformers' object is not iterable